In [1]:
import numpy as np
import random
import struct
import warnings
import os
# Suppress RuntimeWarnings related to overflow
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [2]:
def float_to_bin(num):
    '''Input floating points. Output 32 bits single-percision string in binary'''
    bits, = struct.unpack('!I', struct.pack('!f', num))
    return "{:032b}".format(bits)
def binStr_to_hex(bin_str):
    """Input a binary string. Output a HEX string."""
    # Convert binary string to an integer
    int_value = int(bin_str, 2)
    
    # Convert integer to hexadecimal and remove the '0x' prefix
    hex_value = hex(int_value)[2:]
    
    # Pad with leading zeros to ensure it's 8 characters long (32 bits)
    hex_value = hex_value.zfill(8)  # Ensure it's 8 hex digits (32 bits)
    
    return hex_value
def binStr_to_float32(bin_str):
    """Input a 32 bit binary string. Output a np.float32 number."""
    if len(bin_str) != 32:
        raise ValueError("The binary string must be 32 bits long.")
    # Convert the binary string to an unsigned 32-bit integer
    int_representation = int(bin_str, 2)
    # Convert the integer to a float32
    float_representation = np.float32(np.frombuffer(np.uint32(int_representation).tobytes(), dtype=np.float32)[0])
    return float_representation

SOME CORNER CASES

In [3]:
SPECIAL_LIST = []

POS_NORMAL_UPPERBOUND = np.float32((2-2**(-23))*2**127) 
NEG_NORMAL_UPPERBOUND = np.float32(POS_NORMAL_UPPERBOUND * -1)
SPECIAL_LIST.append(POS_NORMAL_UPPERBOUND)
SPECIAL_LIST.append(NEG_NORMAL_UPPERBOUND)

POS_NORMAL_UPPERBOUND_MINUS = np.float32((2-2**(-23))*2**127 - 2**104) 
NEG_NORMAL_UPPERBOUND_MINUS = np.float32(POS_NORMAL_UPPERBOUND_MINUS * -1)
SPECIAL_LIST.append(POS_NORMAL_UPPERBOUND_MINUS)
SPECIAL_LIST.append(NEG_NORMAL_UPPERBOUND_MINUS)

POS_NORMAL_LOWERBOUND = np.float32(2**(-126)) 
NEG_NORMAL_LOWERBOUND = np.float32(POS_NORMAL_LOWERBOUND * -1)
SPECIAL_LIST.append(POS_NORMAL_LOWERBOUND)
SPECIAL_LIST.append(NEG_NORMAL_LOWERBOUND)

POS_NORMAL_LOWERBOUND_PLUS = np.float32(2**(-126) + 2**(-149)) 
NEG_NORMAL_LOWERBOUND_PLUS = np.float32(POS_NORMAL_LOWERBOUND_PLUS * -1)
SPECIAL_LIST.append(POS_NORMAL_LOWERBOUND_PLUS)
SPECIAL_LIST.append(NEG_NORMAL_LOWERBOUND_PLUS)

POS_SUBNORMAL_UPPERBOUND = np.float32((1-2**(-23)) * 2**(-126)) 
NEG_SUBNORMAL_UPPERBOUND = np.float32(POS_SUBNORMAL_UPPERBOUND * -1)
SPECIAL_LIST.append(POS_SUBNORMAL_UPPERBOUND)
SPECIAL_LIST.append(NEG_SUBNORMAL_UPPERBOUND)

POS_SUBNORMAL_UPPERBOUND_MINUS = np.float32((1-2**(-23)) * 2**(-126) - 2**(-149)) 
NEG_SUBNORMAL_UPPERBOUND_MINUS = np.float32(POS_SUBNORMAL_UPPERBOUND_MINUS * -1)
SPECIAL_LIST.append(POS_SUBNORMAL_UPPERBOUND_MINUS)
SPECIAL_LIST.append(NEG_SUBNORMAL_UPPERBOUND_MINUS)

POS_SUBNORMAL_LOWERBOUND = np.float32(2**(-149)) 
NEG_SUBNORMAL_LOWERBOUND = np.float32(POS_SUBNORMAL_LOWERBOUND * -1)
SPECIAL_LIST.append(POS_SUBNORMAL_LOWERBOUND)
SPECIAL_LIST.append(NEG_SUBNORMAL_LOWERBOUND)

POS_SUBNORMAL_LOWERBOUND_PLUS = np.float32(2**(-149) + 2**(-149)) 
NEG_SUBNORMAL_LOWERBOUND_PLUS = np.float32(POS_SUBNORMAL_LOWERBOUND_PLUS * -1)
SPECIAL_LIST.append(POS_SUBNORMAL_LOWERBOUND_PLUS)
SPECIAL_LIST.append(NEG_SUBNORMAL_LOWERBOUND_PLUS)

POS_ZERO = np.float32(struct.unpack('!f', struct.pack('!I', 0x00000000))[0])
NEG_ZERO = np.float32(struct.unpack('!f', struct.pack('!I', 0x80000000))[0])
SPECIAL_LIST.append(POS_ZERO)
SPECIAL_LIST.append(NEG_ZERO)

POS_NORMAL = np.float32(8787878787878787.878787) 
NEG_NORMAL = np.float32(POS_NORMAL * -1)
SPECIAL_LIST.append(POS_NORMAL)
SPECIAL_LIST.append(NEG_NORMAL)

POS_SUBNORMAL = np.float32(2**-127) 
NEG_SUBNORMAL = np.float32(POS_SUBNORMAL * -1)
SPECIAL_LIST.append(POS_SUBNORMAL)
SPECIAL_LIST.append(NEG_SUBNORMAL)

print(len(SPECIAL_LIST))


22


In [4]:
print((np.float32((NEG_ZERO==NEG_ZERO))))
print((np.float32(POS_ZERO<POS_SUBNORMAL_LOWERBOUND)) == 1)

1.0
True


SOME CORNER CASES: to test double rounding

In [5]:
EXP_STEP = 8
MANTISSA_RANDOM_NUM = 3
# sign bit
SIGN_list = [0, 1]
# exp 
EXP_list = []
exp = 0
while True:
    EXP_list.append(exp)
    exp += EXP_STEP
    if(exp >= 254):
        EXP_list.append(254)
        break
# mantissa
MANTISSA_list = []
for i in range(MANTISSA_RANDOM_NUM):
    random_MANTISSA = np.random.uniform(0, 2**23 - 1)
    MANTISSA_list.append(int(random_MANTISSA))
MANTISSA_list.append(0)
MANTISSA_list.append(2**23 - 1)


PATTERN_TEST_DOUBLE_ROUND = [] # when rounding in floating point arithmetic 進位, need to normalize again
for sign in SIGN_list:
    for exp in EXP_list:
        for mantissa in MANTISSA_list:
            sign_str = ("{:01b}".format(sign))
            exp_str = ("{:08b}".format(exp))
            mantissa_str = ("{:023b}".format(mantissa))
            fp_str = sign_str + exp_str + mantissa_str
            PATTERN_TEST_DOUBLE_ROUND.append(fp_str)

for i in range(len(PATTERN_TEST_DOUBLE_ROUND)):
    PATTERN_TEST_DOUBLE_ROUND[i] = binStr_to_float32(PATTERN_TEST_DOUBLE_ROUND[i])


RANDOMLY GENERATE TEST PATTERN( include some corner cases)

In [6]:
def generate_random_uniform(MIN, MAX):
    """randomly generate float32 number in uniform distribution"""
    random_pos = np.random.uniform(np.float32(MIN), np.float32(MAX))
    random_sign = random.choice([-1, 1])
    random_float32 = np.float32((random_pos * random_sign))
    return np.float32(random_float32)

In [7]:
def generate_random_log(MIN, MAX):
    """randomly generate float32 number in log distribution"""
    # Constants representing the range limits in log space
    min_log = np.log(MIN).astype(np.float32)
    max_log = np.log(MAX).astype(np.float32)
    # Generate a random float32 in log space, then exponentiate it
    random_log_value = np.random.uniform(min_log, max_log)
    random_pos = np.exp(random_log_value).astype(np.float32)
    random_sign = random.choice([-1, 1])
    random_float32 = np.float32((random_pos * random_sign))
    return np.float32(random_float32)

In [8]:
# Function to generate 10 pairs of random single-precision numbers between -10 and 10
def generate_random_pattern_list(INST='ADD', TOTAL_PATTERN_NUM = 1000, SPECIAL_NUM = 0, 
                                 UNIFORM_SUBNORM_NUM=0, UNIFORM_NORM_NUM=0, LOG_SUBNORM_NUM=0, LOG_NORM_NUM=0, 
                                 probabilities = [0.005, 0.005,  0.2475, 0.2475, 0.2475, 0.2475]):
    
    """RANDOMLY GENERATE PATTERN LIST"""
    # lists of np.float32 numbers
    pattern_A_list = []
    pattern_B_list = []
    pattern_OUT_list = []

    for i in range(SPECIAL_NUM):
        for j in range(SPECIAL_NUM):
            pattern_A_list.append(SPECIAL_LIST[i])
            pattern_B_list.append(SPECIAL_LIST[j])
            if INST == 'ADD':
                pattern_OUT_list.append(SPECIAL_LIST[i] + SPECIAL_LIST[j])
            elif INST == 'SUB':
                pattern_OUT_list.append(SPECIAL_LIST[i] - SPECIAL_LIST[j])
            elif INST == 'CMP':
                pattern_OUT_list.append(np.float32(SPECIAL_LIST[i] < SPECIAL_LIST[j])) #1: true, 0: false

    
    """for i in range(len(PATTERN_TEST_DOUBLE_ROUND)):
        for j in range(len(PATTERN_TEST_DOUBLE_ROUND)):
            pattern_A_list.append(PATTERN_TEST_DOUBLE_ROUND[i])
            pattern_B_list.append(PATTERN_TEST_DOUBLE_ROUND[j])
            if INST == 'ADD':
                pattern_OUT_list.append(PATTERN_TEST_DOUBLE_ROUND[i] + PATTERN_TEST_DOUBLE_ROUND[j])
            elif INST == 'SUB':
                pattern_OUT_list.append(PATTERN_TEST_DOUBLE_ROUND[i] - PATTERN_TEST_DOUBLE_ROUND[j])
            elif INST == 'CMP':
                pattern_OUT_list.append(np.float32(PATTERN_TEST_DOUBLE_ROUND[i] < PATTERN_TEST_DOUBLE_ROUND[j])) #1: true, 0: false
    """

    # randomly generate 2 SUBNORMAL numbers in UNIFORM distribution
    for _ in range(UNIFORM_SUBNORM_NUM):
        A = generate_random_uniform(MIN = 2**-149, MAX = (1 - 2**-23) * 2**-126)
        B = generate_random_uniform(MIN = 2**-149, MAX = (1 - 2**-23) * 2**-126)
        pattern_A_list.append(A)
        pattern_B_list.append(B)
        if INST == 'ADD':
            pattern_OUT_list.append(A + B)
        elif INST == 'SUB':
            pattern_OUT_list.append(A - B)
        elif INST == 'CMP':
                pattern_OUT_list.append(np.float32(A < B)) #1: true, 0: false
    # randomly generate 2 NORMAL numbers in UNIFORM distribution
    for _ in range(UNIFORM_NORM_NUM):
        A = generate_random_uniform(MIN = 2**-126, MAX = (2 - 2**-23) * 2**127)
        B = generate_random_uniform(MIN = 2**-126, MAX = (2 - 2**-23) * 2**127)
        pattern_A_list.append(A)
        pattern_B_list.append(B)
        if INST == 'ADD':
            pattern_OUT_list.append(A + B)
        elif INST == 'SUB':
            pattern_OUT_list.append(A - B)
        elif INST == 'CMP':
                pattern_OUT_list.append(np.float32(A < B)) #1: true, 0: false
    # randomly generate 2 SUBNORMAL numbers in LOG distribution
    for _ in range(LOG_SUBNORM_NUM):
        A = generate_random_log(MIN = 2**-149, MAX = (1 - 2**-23) * 2**-126)
        B = generate_random_log(MIN = 2**-149, MAX = (1 - 2**-23) * 2**-126)
        pattern_A_list.append(A)
        pattern_B_list.append(B)
        if INST == 'ADD':
            pattern_OUT_list.append(A + B)
        elif INST == 'SUB':
            pattern_OUT_list.append(A - B)
        elif INST == 'CMP':
                pattern_OUT_list.append(np.float32(A < B)) #1: true, 0: false
    # randomly generate 2 NORMAL numbers in LOG distribution
    for _ in range(LOG_NORM_NUM):
        A = generate_random_log(MIN = 2**-126, MAX = (2 - 2**-23) * 2**127)
        B = generate_random_log(MIN = 2**-126, MAX = (2 - 2**-23) * 2**127)
        pattern_A_list.append(A)
        pattern_B_list.append(B)
        if INST == 'ADD':
            pattern_OUT_list.append(A + B)
        elif INST == 'SUB':
            pattern_OUT_list.append(A - B)
        elif INST == 'CMP':
                pattern_OUT_list.append(np.float32(A < B)) #1: true, 0: false
    
    LEN_SO_FAR = len(pattern_A_list)
    if(TOTAL_PATTERN_NUM - LEN_SO_FAR < 0):
        print("ERROR: TOTAL_PATTERN_NUM - LEN_SO_FAR < 0")
    # MIX
    for _ in range(TOTAL_PATTERN_NUM - LEN_SO_FAR):
        num_list_A = [POS_ZERO, NEG_ZERO, 
                    generate_random_uniform(MIN = 2**-149, MAX = (1 - 2**-23) * 2**-126),
                    generate_random_uniform(MIN = 2**-126, MAX = (2 - 2**-23) * 2**127),
                    generate_random_log(MIN = 2**-149, MAX = (1 - 2**-23) * 2**-126),
                    generate_random_log(MIN = 2**-126, MAX = (2 - 2**-23) * 2**127)
        ]
        num_list_B = [POS_ZERO, NEG_ZERO, 
                    generate_random_uniform(MIN = 2**-149, MAX = (1 - 2**-23) * 2**-126),
                    generate_random_uniform(MIN = 2**-126, MAX = (2 - 2**-23) * 2**127),
                    generate_random_log(MIN = 2**-149, MAX = (1 - 2**-23) * 2**-126),
                    generate_random_log(MIN = 2**-126, MAX = (2 - 2**-23) * 2**127)
        ]
        A = np.random.choice(num_list_A, p=probabilities)
        B = np.random.choice(num_list_B, p=probabilities)
        pattern_A_list.append(A)
        pattern_B_list.append(B)
        if INST == 'ADD':
            pattern_OUT_list.append(A + B)
        elif INST == 'SUB':
            pattern_OUT_list.append(A - B)
        elif INST == 'CMP':
                pattern_OUT_list.append(np.float32(A < B)) #1: true, 0: false

    return pattern_A_list, pattern_B_list, pattern_OUT_list

WRITE OUTPUT DATA

In [9]:
# Function to write pairs into a text file
def write_to_txt(INST, pattern_A_list, pattern_B_list, pattern_OUT_list, root_dir = '',  file_dir_A='A.dat', file_dir_B='B.dat', file_dir_OUT='OUT.dat'):
    # A
    with open(os.path.join(root_dir, file_dir_A), 'w') as file:
        for num in pattern_A_list:
            file.write(f"{float_to_bin(num)}  // {binStr_to_hex(float_to_bin(num))}\n")
    # B
    with open(os.path.join(root_dir, file_dir_B), 'w') as file:
        for num in pattern_B_list:
            file.write(f"{float_to_bin(num)}  // {binStr_to_hex(float_to_bin(num))}\n")
    # OUT
    if INST == 'CMP':
        with open(os.path.join(root_dir, file_dir_OUT), 'w') as file:
            for num in pattern_OUT_list:
                if num == 1: #TRUE  
                    file.write(f"00000000000000000000000000000001 // TRUE\n")
                elif num == 0: #FALSE  
                    file.write(f"00000000000000000000000000000000 // FALSE\n")
    elif INST == 'ADD' or INST == 'SUB':
        with open(os.path.join(root_dir, file_dir_OUT), 'w') as file:
            for num in pattern_OUT_list:
                file.write(f"{float_to_bin(num)}  // {binStr_to_hex(float_to_bin(num))}\n")

MAIN

In [11]:
TOTAL_PATTERN_NUM = 100000 #62500
SPECIAL_NUM = 22
UNIFORM_SUBNORM_NUM = 10000
UNIFORM_NORM_NUM = 10000
LOG_SUBNORM_NUM = 20000
LOG_NORM_NUM = 20000


root_dir = "./MY_pattern/ADD"
INST='ADD'
pattern_A_list, pattern_B_list, pattern_OUT_list = generate_random_pattern_list(INST,TOTAL_PATTERN_NUM, SPECIAL_NUM, 
                                 UNIFORM_SUBNORM_NUM, UNIFORM_NORM_NUM, LOG_SUBNORM_NUM, LOG_NORM_NUM)
write_to_txt(INST, pattern_A_list, pattern_B_list, pattern_OUT_list, root_dir)
print("ADD pattern complete")

root_dir = "./MY_pattern/SUB"
INST='SUB'
pattern_A_list, pattern_B_list, pattern_OUT_list = generate_random_pattern_list(INST,TOTAL_PATTERN_NUM, SPECIAL_NUM, 
                                 UNIFORM_SUBNORM_NUM, UNIFORM_NORM_NUM, LOG_SUBNORM_NUM, LOG_NORM_NUM)
write_to_txt(INST, pattern_A_list, pattern_B_list, pattern_OUT_list, root_dir)
print("SUB pattern complete")

root_dir = "./MY_pattern/CMP"
INST='CMP'
pattern_A_list, pattern_B_list, pattern_OUT_list = generate_random_pattern_list(INST,TOTAL_PATTERN_NUM, SPECIAL_NUM, 
                                 UNIFORM_SUBNORM_NUM, UNIFORM_NORM_NUM, LOG_SUBNORM_NUM, LOG_NORM_NUM)
write_to_txt(INST, pattern_A_list, pattern_B_list, pattern_OUT_list, root_dir)
print("CMP pattern complete")

ADD pattern complete
SUB pattern complete
CMP pattern complete
